In [64]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from urllib.parse import urlencode
import csv


In [65]:
# CSV file name
csvFile = "JobList.csv"

# Base URL
domain = "https://cv.ee/et/search?"

# Query parameters
params = {
    "limit": "1300",
    "offset": "0",
    "keywords[0]": "terraform",
    "fuzzy": "true",
    "towns[0]": "312",
    "suitableForRefugees": "false",
    "isRemoteWork": "true",
    "isHourlySalary": "false",
    "isQuickApply": "false",
    "sorting": "LATEST",
}

# Constructing the URL
url = f"{domain}{urlencode(params)}"

# Making the request and parsing the HTML
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")

In [66]:
# Find all elements
job_listing_boxes = soup.find_all(class_="vacancies-list__item")

# Initialize results list
results = []

# Parse all the elements
for item in job_listing_boxes:
    parsed = {}
    if title := item.find(class_="vacancy-item__title"):
        parsed["title"] = title.text
    if item_url := item.a["href"]:
        parsed["url"] = urljoin(url, item_url.split("Name:")[-1])
    if company := item.find(class_="vacancy-item__body").a:
        parsed["company"] = company.text
    if location := item.find(class_="vacancy-item__locations"):
        parsed["location"] = location.text
    if salary := item.find(class_="vacancy-item__salary-label"):
        parsed["salary"] = salary.text
    else: parsed["salary"] = " "
    if expiry := item.find(class_="vacancy-item__expiry"):
        parsed["expiry"] = expiry.text.split("Tähtaeg: ")[-1].strip()
    #if posted_on := item.find("span", class_="color-white-mute", text=re.compile("posted:", re.I)):
    #    parsed["posted_on"] = posted_on.text.split("Posted:")[-1].strip()
    results.append(parsed)

In [67]:
# Print parsed elements
results

[{'title': 'Data & Cloud Engineer',
  'url': 'https://cv.ee/et/vacancy/1134583/solita-ou/data-and-cloud-engineer',
  'company': 'Solita OÜ',
  'location': 'Tartu /  Kaugtöö',
  'salary': ' ',
  'expiry': '15.02.2024'},
 {'title': 'Infrastructure Automation/Configuration Specialist - Platform engineering team',
  'url': 'https://cv.ee/et/vacancy/1133844/luminor/infrastructure-automation-configuration-specialist-platform-engineering-team',
  'company': 'Luminor',
  'location': 'Tallinn /  Kaugtöö',
  'salary': 'Alates \xa0€ 3700',
  'expiry': '21.01.2024'},
 {'title': 'Supply IT Infrastructure Architect  ',
  'url': 'https://cv.ee/et/vacancy/1132993/ericsson-estonia-as/supply-it-infrastructure-architect',
  'company': 'Ericsson Estonia AS',
  'location': 'Tallinn',
  'salary': ' ',
  'expiry': '25.01.2024'},
 {'title': 'DevOps Engineer with focus on Data Science',
  'url': 'https://cv.ee/et/vacancy/1125715/swedbank-as/devops-engineer-with-focus-on-data-science',
  'company': 'Swedbank AS

In [68]:
# Write list to CSV file. 
myFile = open(csvFile, 'w')
writer = csv.writer(myFile)
writer.writerow(['title', 'url', 'company', "location", "salary", "expiry"])
for dictionary in results:
    writer.writerow(dictionary.values())
myFile.close()

In [69]:
# Print CSV file
myFile = open(csvFile, 'r')
print("The content of the csv file is:")
print(myFile.read())
myFile.close()

The content of the csv file is:
title,url,company,location,salary,expiry
Data & Cloud Engineer,https://cv.ee/et/vacancy/1134583/solita-ou/data-and-cloud-engineer,Solita OÜ,Tartu /  Kaugtöö, ,15.02.2024
Infrastructure Automation/Configuration Specialist - Platform engineering team,https://cv.ee/et/vacancy/1133844/luminor/infrastructure-automation-configuration-specialist-platform-engineering-team,Luminor,Tallinn /  Kaugtöö,Alates  € 3700,21.01.2024
Supply IT Infrastructure Architect  ,https://cv.ee/et/vacancy/1132993/ericsson-estonia-as/supply-it-infrastructure-architect,Ericsson Estonia AS,Tallinn, ,25.01.2024
DevOps Engineer with focus on Data Science,https://cv.ee/et/vacancy/1125715/swedbank-as/devops-engineer-with-focus-on-data-science,Swedbank AS,Tallinn /  Kaugtöö,€ 3300 – 5000,21.01.2024

